## Example: using Linear Programming to solve for 3 latent factors `w`

Please note that the last part of this example, "Permutation invariance in linear programming vs. MultiCCA in R and `pmd`" requires that `inpt3` be exactly as printed.

In [1]:
import pandas as pd
import numpy as np

from sparsecca import lp_pmd

In [2]:
# example inputs
inpt1 = pd.read_csv("../tests/data/multicca1.csv", sep=",", index_col=0).values
inpt2 = pd.read_csv("../tests/data/multicca2.csv", sep=",", index_col=0).values[:, -5:]
inpt3 = np.random.normal(size=inpt2.shape)

penalties = [1.5, 1.5, 1.5]

In [34]:
inpt3

array([[ 1.16289006, -1.10155538, -0.07361009, -1.04768133, -0.23030313],
       [-0.28524699, -0.95404835,  1.25754511,  2.22141375, -0.64806753],
       [-0.80371799, -0.09921252, -1.30902827, -0.12490934, -0.53780387],
       [ 1.23123296,  1.30839663,  2.35443782,  1.2004608 , -0.24753196],
       [ 1.24686808,  0.2176884 , -0.51074504,  1.68211628, -0.0589116 ],
       [-0.54485555, -0.31221068, -0.72426958,  0.11262969,  0.06324836],
       [-1.36002346, -2.02609761, -0.77052108, -1.47716523, -0.98507913],
       [ 2.01516562, -0.35919419,  0.98862972, -0.30120967, -1.85093266],
       [-1.43260253, -0.00428044, -0.31457342,  0.72451216,  0.59311224],
       [-0.46268311,  2.11013054, -0.80026669,  0.03718434, -0.66038343],
       [ 1.79330705, -1.77072566,  0.59342351,  0.5705045 , -0.26306724],
       [-0.53183765,  0.46836312, -0.16679474,  0.24751673,  0.48850261],
       [ 0.64762214, -0.2314719 , -0.00601488, -1.31931633, -0.02510419],
       [-0.0595952 , -1.14180706,  0.1

In [3]:
weights, svd_init = lp_pmd(
    datasets=[inpt1, inpt2], # match feature dimension for now
    penalties=penalties[:2],
    K=3,
    standardize=True,
    mimic_R=True
)

In [4]:
weights

array([[[-0.08911592,  0.05447262, -0.17954319],
        [ 0.66470561, -0.54663329,  0.6135666 ],
        [ 0.60794117, -0.65622806,  0.55246482],
        [-0.42496254,  0.42511018, -0.52736503],
        [-0.0062595 , -0.29473863, -0.08926939]],

       [[ 0.49615515, -0.43360575,  0.50388569],
        [ 0.30569572, -0.57468281,  0.2926027 ],
        [-0.13754639, -0.06168402, -0.09398377],
        [-0.73325299,  0.47846533, -0.74173363],
        [ 0.32218201, -0.49899073,  0.31856108]]])

## Correlation between pmd results and lp results

In [5]:
from scipy.stats import spearmanr
from sparsecca import multicca_pmd

### high correlation between pmd and lp when `n=2`

Let `n` equal the number of datasets.

In [6]:
datasets = [inpt1, inpt2]
ws_lp, _ = lp_pmd(datasets, penalties[:2])
ws_r, _ = multicca_pmd(datasets, penalties[:2])

In [7]:
print(spearmanr(ws_lp[0], ws_r[0]))
print(spearmanr(ws_lp[1], ws_r[1]))

SignificanceResult(statistic=0.9746794344808964, pvalue=0.004818230468198537)
SignificanceResult(statistic=0.8999999999999998, pvalue=0.03738607346849874)


Note that this is the same as taking the first latent factor when K=3, as per the implementation in Witten 2009.

In [9]:
print(spearmanr(weights[0].T[0], ws_r[0]))
print(spearmanr(weights[1].T[0], ws_r[1]))

SignificanceResult(statistic=0.9746794344808964, pvalue=0.004818230468198537)
SignificanceResult(statistic=0.8999999999999998, pvalue=0.03738607346849874)


### arbitrarily lower correlation when `n=3`

Compare when `n=3`. Note that this is subject to the same warning as in the bottom of the notebook.

In [10]:
datasets = [inpt1, inpt2, inpt3]
ws_lp, _ = lp_pmd(datasets, penalties[:3])
ws_r, _ = multicca_pmd(datasets, penalties[:3])

In [11]:
print(spearmanr(ws_lp[0], ws_r[0]))
print(spearmanr(ws_lp[1], ws_r[1]))
print(spearmanr(ws_lp[2], ws_r[2]))

SignificanceResult(statistic=-0.8720815992723809, pvalue=0.05385421772754211)
SignificanceResult(statistic=-0.35909242322980395, pvalue=0.5528147466433505)
SignificanceResult(statistic=-0.09999999999999999, pvalue=0.8728885715695383)


#### Optimization of the objective function

Size of the objective function, which was maximized:

In [17]:
print('1st pair:', ws_lp[0].T @ inpt1.T @ inpt2 @ ws_lp[1])
print('2nd pair:', ws_lp[1].T @ inpt2.T @ inpt3 @ ws_lp[2])
print('3rd pair:', ws_lp[2].T @ inpt3.T @ inpt1 @ ws_lp[0])
print('sum', ws_lp[0].T @ inpt1.T @ inpt2 @ ws_lp[1] + ws_lp[1].T @ inpt2.T @ inpt3 @ ws_lp[2] + ws_lp[2].T @ inpt3.T @ inpt1 @ ws_lp[0])

1st pair: [[111.69927657]]
2nd pair: [[16.09054608]]
3rd pair: [[29.41034686]]
sum [[157.20016951]]


In [18]:
print('1st pair:', ws_r[0].T @ inpt1.T @ inpt2 @ ws_r[1])
print('2nd pair:', ws_r[1].T @ inpt2.T @ inpt3 @ ws_r[2])
print('3rd pair:', ws_r[2].T @ inpt3.T @ inpt1 @ ws_r[0])
print('sum', ws_r[0].T @ inpt1.T @ inpt2 @ ws_r[1] + ws_r[1].T @ inpt2.T @ inpt3 @ ws_r[2] + ws_r[2].T @ inpt3.T @ inpt1 @ ws_r[0])

1st pair: [[108.09846793]]
2nd pair: [[21.92976799]]
3rd pair: [[30.2407647]]
sum [[160.26900062]]


L2 norm of the latent factors, constrained.

In [19]:
np.linalg.norm(ws_lp)

1.7320508160371626

In [20]:
np.linalg.norm(ws_r)

1.7320508075688772

Neither linear programming nor manual convergence consistently produces a better optimization of the objective function in this small test case.

## Permutation invariance in linear programming vs. MultiCCA in R and `pmd`

In [21]:
def test_weights(weights_a, weights_b, perm_b: list[int], dec=5):
    """Tests whether `weights_a` and `weights_b` are the same given the permutation order of b.

    Parameters:
        weights_a: output of lp_pmd 
        weights_b: output of lp_pmd permuted Xn
                   -> weights are of type np.ndarray in shape (N, f, K)
                    - N: len(Xn) datasets
                    - f: amount of features
                    - K: amount of MCPs
        perm_b:    order of the datasets used to generate a, in b
        dec:       decimals to which weights should be rounded to account for numerical tolerance

    Returns:
        boolean: True if rounded weights are the same, else False
    """
    
    weights_a_rounded = weights_a.round(decimals=dec)
    weights_b_rounded = weights_b.round(decimals=dec)
    
    weights_b_ordered = []
    for o in perm_b:
        weights_b_ordered.append(weights_b_rounded[o])
        
    return all(x==True for x in (weights_a_rounded==weights_b_ordered).flatten())

In [22]:
datasets = [inpt1, inpt2, inpt3]
# original dataset with perm 1, 2, 0
datasets_perm = [inpt3, inpt1, inpt2]

### linear programming

In [24]:
ws_lp, _ = lp_pmd(datasets, penalties)
ws_lp_perm, _ = lp_pmd(datasets_perm, penalties)

In [25]:
ws_lp

array([[[-0.06329702],
        [-0.47883099],
        [-0.71653621],
        [ 0.50235555],
        [-0.03048172]],

       [[-0.35167889],
        [-0.48543111],
        [-0.07157119],
        [ 0.54892933],
        [-0.57812867]],

       [[-0.74880871],
        [-0.52826939],
        [-0.39417701],
        [ 0.00182197],
        [ 0.06955682]]])

In [26]:
ws_lp_perm

array([[[-0.74880871],
        [-0.52826939],
        [-0.39417701],
        [ 0.00182197],
        [ 0.06955682]],

       [[-0.06329702],
        [-0.47883099],
        [-0.71653621],
        [ 0.50235555],
        [-0.03048172]],

       [[-0.35167889],
        [-0.48543111],
        [-0.07157119],
        [ 0.54892933],
        [-0.57812867]]])

In [27]:
test_weights(ws_lp, ws_lp_perm, [1,2,0], dec=5)

True

As we can see, the weights are the same with merely the order permuted as is appropriate.

### R (in python)

In [28]:
ws_r, _ = multicca_pmd(datasets, penalties)
ws_r_perm, _ = multicca_pmd(datasets_perm, penalties)

In [29]:
ws_r

[array([[-0.        ],
        [ 0.8315665 ],
        [ 0.54048778],
        [-0.12794572],
        [-0.        ]]),
 array([[ 0.72935664],
        [-0.0928381 ],
        [-0.        ],
        [-0.67780527],
        [ 0.        ]]),
 array([[ 0.        ],
        [ 0.89010769],
        [-0.42640703],
        [-0.15901786],
        [ 0.02446747]])]

In [30]:
ws_r_perm

[array([[ 0.12178605],
        [ 0.93963595],
        [-0.27441203],
        [-0.16416603],
        [ 0.        ]]),
 array([[-0.        ],
        [ 0.75371432],
        [ 0.65012982],
        [-0.09615587],
        [-0.        ]]),
 array([[ 0.73184218],
        [-0.        ],
        [-0.        ],
        [-0.67508905],
        [ 0.09306875]])]

Test with decimal tolerance 5.

In [31]:
test_weights(np.array(ws_r), np.array(ws_r_perm), [1,2,0])

False

Test with decimal tolerance 1.

In [32]:
test_weights(np.array(ws_r), np.array(ws_r_perm), [1,2,0], dec=1)

False

Test the equivalent negative solution.

In [33]:
test_weights(np.array(ws_r), -np.array(ws_r_perm), [1,2,0], dec=1)

False

**Note**: linear programming will always solve for the same objective function regardless of the order of the inputs, but the custom PMD implementation in `multicca_pmd` will not. Depending on the dataset, the solution may converge similarly regardless of order (with only a difference in sign), or the solution may converge to a completely different local minima given a different order.